In [286]:
##############
#Dependencies#
##############
%matplotlib notebook

import pandas as pd
import gmaps
import numpy as np
from config import (gkey, 
#                     route)
                   )

In [230]:
gmaps.configure(api_key=gkey)

In [270]:
route = ['S567','S583','S568','S582','S569','S570','S581',
'S571','S579','S572','S578','S573',"S577","S587"]
station_df = pandas.read_csv('data/station_config.csv')
route_df= pandas.read_csv('data/route_summary.csv')
station_df = station_df.set_index("Station")
route_df = route_df.set_index("Station")
route_df= route_df.replace("UNKNOWN",1)
route_df.head()

,Station.1,Time,Occupancy,Speed,Flow
Station,,,,,
S567,NaN,Sat Oct 06 22:09:29 CDT 2018,3.54,64,520
S583,NaN,Sat Oct 06 22:09:29 CDT 2018,4.67,56,570
S568,NaN,Sat Oct 06 22:09:29 CDT 2018,3.61,48,480
S582,NaN,Sat Oct 06 22:09:29 CDT 2018,4.31,74,760
S569,NaN,Sat Oct 06 22:09:29 CDT 2018,3.86,78,780


In [271]:

Results = {}


locations = []

for station in route:
        Flow =  int(route_df.loc[station,'Flow'].tail(1)[0])
        Speed = int(route_df.loc[station,'Speed'].tail(1)[0])

        Max_Flow = route_df.loc[station,'Flow'].dropna()
        Max_Flow = int(max(Max_Flow.values))
        Max_Speed = route_df.loc[station,'Speed'].dropna()
        Max_Speed = int(max(Max_Speed.values))
        
        lat, lon = station_df.loc[station, ['Lat', 'Lng']]
        locations.append((lat, lon))
    
        Results[station] = {
            "Current Speed" : Speed,
            "Current Flow" : Flow,
            "Max Speed" : Max_Speed,
            "Max Flow" : Max_Flow,
            "% of Max Speed" : round(Speed / Max_Speed * 100, 2),
            "% of Max Flow" : round(Flow / Max_Flow * 100, 2) ,
            "Lat":lat, 
            "Lon":lon}

In [272]:
Results = pd.DataFrame(Results)
Results.head()
Results

,S567,S583,S568,S582,S569,S570,S581,S571,S579,S572,S578,S573,S577,S587
% of Max Flow,100.00000,54.17000,75.00000,78.26000,58.62000,54.55000,66.67000,61.90000,50.00000,60.00000,100.00000,66.67000,86.36000,77.78000
% of Max Speed,89.55000,84.06000,76.67000,87.80000,91.21000,87.50000,90.54000,83.08000,97.33000,73.56000,90.16000,93.51000,97.10000,93.55000
Current Flow,880.00000,390.00000,720.00000,720.00000,510.00000,480.00000,640.00000,520.00000,400.00000,480.00000,840.00000,480.00000,760.00000,420.00000
Current Speed,60.00000,58.00000,46.00000,72.00000,83.00000,77.00000,67.00000,54.00000,73.00000,64.00000,55.00000,72.00000,67.00000,58.00000
Lat,44.97447,44.98175,44.98399,44.99115,44.99116,44.99644,44.99646,45.00199,45.00091,45.00070,45.00302,45.00515,45.01014,45.00891
Lon,-93.24870,-93.24397,-93.24197,-93.23659,-93.23629,-93.23823,-93.23853,-93.23465,-93.22346,-93.22349,-93.21487,-93.20947,-93.20040,-93.20233
Max Flow,880.00000,720.00000,960.00000,920.00000,870.00000,880.00000,960.00000,840.00000,800.00000,800.00000,840.00000,720.00000,880.00000,540.00000
Max Speed,67.00000,69.00000,60.00000,82.00000,91.00000,88.00000,74.00000,65.00000,75.00000,87.00000,61.00000,77.00000,69.00000,62.00000


In [278]:
Results["S567"]['Lat']

44.974470000000004

In [311]:
# Create bank symbol layer
fig = gmaps.figure()

station_layer = gmaps.symbol_layer(locations)

fig.add_layer(station_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [312]:
Features = []
col2lat = ""
col2lon = ""
for col in Results:
    if col2lat == '' :
        col2lat = Results[col]['Lat']
        col2lon = Results[col]['Lon']
    else:
        Traffic_Line = gmaps.Line(
            start=(Results[col]['Lat'], Results[col]['Lon']),
            end=(col2lat, col2lon),
            stroke_weight=Results[col]['Current Flow'] / 30
            )
        Features.append(Traffic_Line)
        col2lat = Results[col]['Lat']
        col2lon = Results[col]['Lon']

    
drawing = gmaps.drawing_layer(features=Features)
 
fig.add_layer(drawing)
fig

Figure(layout=FigureLayout(height='420px'))

45.009